# Do interface design to further filter out unproductive designs prior to looping

In [1]:
# %load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers
running on node: sofia


In [2]:
os.chdir("/home/broerman/projects/crispy_shifty")
sys.path.insert(0, os.getcwd())

In [4]:
# testing

sys.path.insert(0, "/home/broerman/projects/crispy_shifty")
from crispy_shifty.protocols.looping import connect_chains

pyrosetta.distributed.maybe_init(**{
    "options": "-corrections::beta_nov16 true",
    "extra_options":{
        "-out:level": "300",
        "-holes:dalphaball": "/home/bcov/ppi/tutorial_build/main/source/external/DAlpahBall/DAlphaBall.gcc",
        "-indexed_structure_store:fragment_store": "/home/bcov/sc/scaffold_comparison/data/ss_grouped_vall_all.h5",
        "-precompute_ig": "true"
}})

from copy import deepcopy
import pyrosetta.distributed.io as io
import pyrosetta.distributed.cluster as cluster
file = '/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter/decoys/0000/02_design_filter_0a0e27de4de64d10b9c4272024e98bf8.pdb'

with open(file, "r") as f:
    pose = io.to_pose(io.pose_from_pdbstring(f.read()))
scores = cluster.get_scores_dict(file)["scores"]

# get parent from packed_pose_in, get loop length from parent length - packed_pose_in length
parent_length = int(float(scores["parent_length"]))
length = int(parent_length - pose.chain_end(2))

sw = pyrosetta.rosetta.protocols.simple_moves.SwitchChainOrderMover()
sw.chain_order("12")
pose_a = deepcopy(pose)
sw.apply(pose_a)
pose_b = deepcopy(pose)
sw.chain_order("34")
sw.apply(pose_b)

for pose, chain in zip([pose_a, pose_b], ["a", "b"]):
    connect_chains(pose, length)
    pose.dump_pdb(f'testing/pose_{chain}')

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: beta_nov16.wts
protocols.simple_moves.SwitchChainOrderMover: Number of chains in pose: 4
protocols.simple_moves.SwitchChainOrderMover: Now at chain: 1
protocols.simple_moves.SwitchChainOrderMover: Now at chain: 2
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/pyrosetta/database/rotamer/beta_nov2016/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/pyrosetta/database/rotamer/beta_nov2016/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.22417 seconds to load from binary
protocols.simple_moves.Switc


ERROR: No loop closures found below threshold, exiting
ERROR:: Exit from: /home/benchmark/rosetta/source/src/protocols/pose_length_moves/ConnectChainsMover.cc line: 221


RuntimeError: 

File: /home/benchmark/rosetta/source/src/protocols/pose_length_moves/ConnectChainsMover.cc:221
[ ERROR ] UtilityExitException
ERROR: No loop closures found below threshold, exiting

